In [2]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

In [3]:
# Number of classes for the food recognition model
num_of_foods = 12

# Load the InceptionV3 model pre-trained on ImageNet, without the top layer
base_model = InceptionV3(
    weights='imagenet',        # Use weights pre-trained on ImageNet
    include_top=False,         # Do not include the top (fully connected) layers
    input_shape=(224, 224, 3)  # Input shape must match InceptionV3's requirements
)

# Freeze the base model to prevent its weights from being updated during training
base_model.trainable = False

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 20s 0us/step


In [4]:
# Functional API approach to build the model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)  # Pass inputs through the InceptionV3 model

# Add global average pooling to reduce dimensions
x = layers.GlobalAveragePooling2D()(x)

# Add a dense layer for learning
x = layers.Dense(128, activation='relu')(x)

# Output layer with softmax activation for multi-class classification
outputs = layers.Dense(num_of_foods, activation='softmax')(x)

# Create the model
model = tf.keras.Model(inputs, outputs)

In [5]:
# Functional API approach to build the model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)  # Pass inputs through the InceptionV3 model

# Add global average pooling to reduce dimensions
x = layers.GlobalAveragePooling2D()(x)

# Add a dense layer for learning
x = layers.Dense(128, activation='relu')(x)

# Output layer with softmax activation for multi-class classification
outputs = layers.Dense(num_of_foods, activation='softmax')(x)

# Create the model
model = tf.keras.Model(inputs, outputs)

In [6]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' if labels are integers
    metrics=['accuracy']
)

# Display the model summary
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_v3 (Functional)       │ (None, 5, 5, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,066,604 (84.18 MB)

 Trainable params: 263,820 (1.01 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [7]:
# Create data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,       # Rescale pixel values from [0, 255] to [0, 1]
    rotation_range=20,    # Data augmentation options
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Use 20% of the data for validation
)

# Path to the training dataset directory
data_dir = 'D:/food_dataset'  # Replace with the actual path

# Create training data generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Resize images to match InceptionV3 input size
    batch_size=32,
    class_mode='sparse',     # Use 'sparse' if labels are integers
    subset='training'
)

# Create validation data generator
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

Found 5038 images belonging to 12 classes.
Found 1254 images belonging to 12 classes.


In [8]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # You can change the number of epochs based on your requirements
    validation_data=validation_generator
)

Epoch 1/10


D:\apps\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


158/158 ━━━━━━━━━━━━━━━━━━━━ 1669s 10s/step - accuracy: 0.6196 - loss: 1.2608 - val_accuracy: 0.8110 - val_loss: 0.5415
Epoch 2/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 1483s 9s/step - accuracy: 0.8317 - loss: 0.4960 - val_accuracy: 0.8596 - val_loss: 0.4467
Epoch 3/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 1476s 9s/step - accuracy: 0.8662 - loss: 0.3956 - val_accuracy: 0.8740 - val_loss: 0.3902
Epoch 4/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 1485s 9s/step - accuracy: 0.8852 - loss: 0.3647 - val_accuracy: 0.8748 - val_loss: 0.4090
Epoch 5/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 1628s 10s/step - accuracy: 0.8825 - loss: 0.3650 - val_accuracy: 0.8971 - val_loss: 0.3132
Epoch 6/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 1547s 9s/step - accuracy: 0.8850 - loss: 0.3440 - val_accuracy: 0.8971 - val_loss: 0.3338
Epoch 7/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 1527s 9s/step - accuracy: 0.8947 - loss: 0.3261 - val_accuracy: 0.8947 - val_loss: 0.3176
Epoch 8/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 1824s 11s/step - accuracy: 0.9033 - loss: 0.2994 - val_accu

In [9]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')
6

40/40 ━━━━━━━━━━━━━━━━━━━━ 397s 10s/step - accuracy: 0.8758 - loss: 0.3912
Validation Accuracy: 88.04%


6